In [ ]:
import panel as pn
pn.extension()

The ``ChatFeed`` allows you to build custom chat interfaces. For example a chat interface for multiple users or a chat interface using buttons as inputs.

See [``ChatInterface``](ChatInterface.ipynb) for a pre-built ChatGPT chat interface.

![Chat Design Specification](../../assets/ChatDesignSpecification.png)

#### Parameters:

For details on other options for customizing the component see the [layout](../../how_to/layout/index.md) and [styling](../../how_to/styling/index.md) how-to guides.

##### Core

* **`value`** (list): The entries added to the chat feed.
* **`header`** (string or viewable): The header of the chat feed.
* **`callback`** (callable): Callback to execute when a user sends a message or
when `respond` is called. The signature must include
the previous message value `contents`, the previous `user` name,
and the component `instance`.

* **`callback_user`** (str): The default user name to use for the entry provided by the callback.
* **`callback_avatar`** (str | BinaryIO): The avatar to use for the user. Can be a single character text, an emoji, or anything supported by `pn.pane.Image`. If not set, uses the first character of the name.
* **`placeholder`** (any): Placeholder to display while the callback is running.
Defaults to a LoadingSpinner.
* **`placeholder_text`** (any): If placeholder is the default LoadingSpinner,
the text to display next to it.
* **`placeholder_threshold`** (float): Min duration in seconds of buffering before displaying the placeholder. If 0, the placeholder will be disabled. Defaults to 0.2.
* **`auto_scroll_limit`** (int): Max pixel distance from the latest object in the Column to activate automatic scrolling upon update. Setting to 0
disables auto-scrolling.
* **`auto_scroll_limit`** (int): Min pixel distance from the latest object in the Column to display the scroll button. Setting to 0 disables the scroll button.
* **`view_latest`** (bool): Whether to scroll to the latest object on init. If not enabled the view will be on the first object. Defaults to True.
* **`card_params`** (dict): Params to pass to Card, like `header`,
`header_background`, `header_color`, etc.
* **`entry_params`** (dict): Params to pass to each ``ChatEntry``, like `reaction_icons`, `timestamp_format`, `show_avatar`, `show_user`, and `show_timestamp`.


* **`reactions`** (List): Reactions associated with the message.
* **`reaction_icons`** (ChatReactionIcons | dict): A mapping of reactions to their reaction icons; if not provided defaults to `{"favorite": "heart"}`. Provides a visual representation of reactions.
* **`timestamp`** (datetime): Timestamp of the message. Defaults to the instantiation time.
* **`timestamp_format`** (str): The format in which the timestamp should be displayed.

##### Display

* **`show_avatar`** (bool): Whether to display the avatar of the user.
* **`show_user`** (bool): Whether to display the name of the user.
* **`show_timestamp`** (bool): Whether to display the timestamp of the message.
* **`name`** (str): The title or name of the chat entry widget, if any.

___

___

In [ ]:
from asyncio import sleep

from panel.widgets.chat import ChatEntry, ChatFeed

pn.extension()

async def get_response(contents, user, instance):
    await sleep(1)
    return {
        "Philipp": None,  # Ignore Philipp :-)
        "Marc": "It is 2",
        "Andrew": "It is 4",
    }.get(user, "I don't know")

ASSISTANT_AVATAR = "https://upload.wikimedia.org/wikipedia/commons/6/60/Icons8_flat_assistant.svg"

chat_feed = ChatFeed(
    value=[ChatEntry(user="Assistant", value="Hello World", avatar=ASSISTANT_AVATAR)],
    callback=get_response,
    height=500,
    callback_avatar=ASSISTANT_AVATAR,
)

marc_button = pn.widgets.Button(
    name="Marc",
    on_click=lambda event: chat_feed.send(
        user="Marc", value="What is the square root of 4", avatar="🚴"
    ),
    align="center",
)
andrew_button = pn.widgets.Button(
    name="Andrew",
    on_click=lambda event: chat_feed.send(
        user="Andrew", value="What is the square root of 4 squared?", avatar="🧑"
    ),
    align="center",
)
philipp_button = pn.widgets.Button(
    name="Philipp",
    on_click=lambda event: chat_feed.send(
        user="Philipp", value="What is 2+2", avatar="🏃"
    ),
    align="center",
)

pn.Column(
    chat_feed,
    pn.layout.Divider(),
    pn.Row("Click to ask a question", marc_button, andrew_button, philipp_button),
).servable()


Note how we use emojis and images (via urls) as avatars in the `ChatFeed`.

We chose to use an *async* callback in the example. We recommend using *async* callbacks and *async* functions from `openai` and other AI frameworks whenever possible for optimal performance.